In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
from glob import glob

import sklearn
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
import pickle

In [28]:
# One-time data augmentation
# Move half of the test data to train -> train:test from 1:1 to 3:1
import shutil
def train_data_augmentation():
    # move the last 30 examples from test to train
    for each_class_path in all_class_path_test_new:
        print(each_class_path)
        for each_class in os.listdir(each_class_path):
            print(os.path.join(each_class_path,each_class))
            for i in os.listdir(os.path.join(each_class_path,each_class)):
                if (int(i)>=30):
                    print(int(i)+30)
                    source = os.path.join(each_class_path,each_class,i)
                    target = os.path.join(each_class_path.replace("newTest","newTrain"),each_class,str(int(i)+30))
                    print("source:",source)
                    print("target:",target)
                    if os.path.exists(source):
                        dest = shutil.move(source, target)
                        print(dest)

    print("Data reorg is done.")

## 1. Process data from MediaPipe Outputs

In [2]:
folder_path = "."
data_path = os.path.join(folder_path, "Data")
model_path = os.path.join(folder_path, "Model")
# path health check
assert os.path.exists(data_path) == True, "data_path not exist: "+data_path
assert os.path.exists(model_path) == True, "model_path not exist: "+model_path

# train_data = os.path.join(data_path,"Train Data Augmentation") 
# test_data = os.path.join(data_path,"Test Data Augmentation") 
train_data = os.path.join(data_path,"newTrain") 
test_data = os.path.join(data_path,"newTest")

In [3]:
all_class_path_train = glob(os.path.join(train_data,"*"))
all_class_path_test = glob(os.path.join(test_data,"*"))
num_classes = 5
print(all_class_path_train)
print("num_classes",num_classes)
print(all_class_path_test)
# train_data_augmentation()

['.\\Data\\newTrain\\CCW Rotate 15', '.\\Data\\newTrain\\CW Rotate 15', '.\\Data\\newTrain\\Left', '.\\Data\\newTrain\\Right', '.\\Data\\newTrain\\Zoom In', '.\\Data\\newTrain\\Zoom Out']
num_classes 5
['.\\Data\\newTest\\CCW Rotate 15', '.\\Data\\newTest\\CW Rotate 15', '.\\Data\\newTest\\Left', '.\\Data\\newTest\\Right', '.\\Data\\newTest\\Zoom In', '.\\Data\\newTest\\Zoom Out']


In [4]:
class_list = ['Kiss', 'Left_punch', 'No_action', 'Right_punch', 'Wakanda']
map_idx2class = {v: k for v, k in enumerate(class_list)}
map_class2idx = { k:v for v, k in enumerate(class_list)}
map_class2idx

{'Kiss': 0, 'Left_punch': 1, 'No_action': 2, 'Right_punch': 3, 'Wakanda': 4}

In [5]:
import time

def load_data(all_class_path, no_face=False):
    X_train = []
    y_train = []

    start = time.time()
    # generate X_train, y_train from all the .npy files
    for each_class_path in all_class_path:
        print(each_class_path)
        for name in os.listdir(each_class_path):
            print("class_name: "+name)
            class_list = os.listdir(each_class_path+"/"+name)
            for i in range(len(class_list)):
                npylst = os.listdir(each_class_path+"/"+name+"/"+class_list[i])
                for j in range(len(npylst)):
                    if os.path.splitext(npylst[j])[1] == '.npy': # 筛选npy文件
                        if no_face == False:
                            arr =  np.load(each_class_path+"/"+name+"/"+class_list[i]+'/'+ npylst[j])
                        else:
                            arr_full =  np.load(each_class_path+"/"+name+"/"+class_list[i]+'/'+ npylst[j])
                            arr = np.concatenate([arr_full[0:132],arr_full[1536:]])
                        X_train.append( arr)
                        y_train.append(map_class2idx.get(name))
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    print("Finish preparing X_train")
    end = time.time()
    print("Time used: ",end - start)
    print(X_train.shape)  
    print("6 * 5 * 60 * 30 = "+ str(6 * 5 * 60 * 30))
    print(y_train.shape)
    return X_train, y_train

In [18]:
print("######################################## Start preparing TRAINING data ######################################## ")
X_train, y_train = load_data(all_class_path_train,no_face=True)
print("######################################## Start preparing TEST data ######################################## ")
X_test, y_test = load_data(all_class_path_test,no_face=True)

######################################## Start preparing TRAINING data ######################################## 
.\Data\newTrain\CCW Rotate 15
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
.\Data\newTrain\CW Rotate 15
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
.\Data\newTrain\Left
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
.\Data\newTrain\Right
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
.\Data\newTrain\Zoom In
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
.\Data\newTrain\Zoom Out
class_name: Kiss
class_name: Left_punch
class_name: No_action
class_name: Right_punch
class_name: Wakanda
Finish preparing X_train
Time used:  26.489355325698853
(81000, 258)
6 * 5 * 60 * 30 = 54000
(81000,

## 2.Build Models

In [27]:
(81000+27000)/2

54000.0

## 2.1 KNN

In [8]:
from sklearn.neighbors import KNeighborsClassifier 

k = num_classes
start = time.time()
KNN = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
Pred_y = KNN.predict(X_train)

print("Training Accuracy of model at K=5 is",metrics.accuracy_score( y_train,Pred_y))
y_pred = KNN.predict(X_test)
print("Test Accuracy of model at K=5 is",metrics.accuracy_score( y_test,y_pred))
print("Time used: ",time.time() - start)

print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 
get_prediction_time(KNN, X_test)

Training Accuracy of model at K=5 is 0.9866543209876544
Test Accuracy of model at K=5 is 0.9398148148148148
Time used:  135.81224632263184
[[5285    0   14    0  101]
 [  11 5297   92    0    0]
 [ 298  238 4458  195  211]
 [   0   66   21 5313    0]
 [ 229    3   74   72 5022]]
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5400
           1       0.95      0.98      0.96      5400
           2       0.96      0.83      0.89      5400
           3       0.95      0.98      0.97      5400
           4       0.94      0.93      0.94      5400

    accuracy                           0.94     27000
   macro avg       0.94      0.94      0.94     27000
weighted avg       0.94      0.94      0.94     27000



In [9]:
get_prediction_time(KNN, X_test)

Average Prediction Time:  0.05199146270751953


In [13]:
# save the model to disk
pickle.dump(KNN, open(os.path.join(model_path, "KNN_Final"), 'wb'))

### 2.2 Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

start = time.time()
rf_clf = RandomForestClassifier(n_estimators=500, random_state=0)  
rf_clf.fit(X_train, y_train)  
y_pred = rf_clf.predict(X_test) 

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(metrics.accuracy_score(y_test, y_pred))  
get_prediction_time(rf_clf, X_test)
print("Time used: ",time.time() - start)

[[5263    0   37    0  100]
 [   0 5229  171    0    0]
 [ 139  127 5029   48   57]
 [   0   72   96 5232    0]
 [   0    0  311   66 5023]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      5400
           1       0.96      0.97      0.97      5400
           2       0.89      0.93      0.91      5400
           3       0.98      0.97      0.97      5400
           4       0.97      0.93      0.95      5400

    accuracy                           0.95     27000
   macro avg       0.96      0.95      0.95     27000
weighted avg       0.96      0.95      0.95     27000

0.9546666666666667
Average Prediction Time:  0.040096092224121097
Time used:  408.5349564552307


In [14]:
# save the model to disk
pickle.dump(rf_clf, open(os.path.join(model_path, "RandomForest_Final"), 'wb'))

### 2.3 AdaBoosting

In [21]:
from sklearn.ensemble import AdaBoostClassifier

start = time.time()
ada = AdaBoostClassifier(n_estimators=60,learning_rate=1)
ada.fit(X_train, y_train)  
y_pred = ada.predict(X_test) 

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(metrics.accuracy_score(y_test, y_pred))
get_prediction_time(ada, X_test)
print("Time used: ",time.time() - start)

[[4666    0  632    0  102]
 [   0 4580  579   17  224]
 [ 898   80 4171  118  133]
 [   1   78  691 4567   63]
 [1255    0  696   77 3372]]
              precision    recall  f1-score   support

           0       0.68      0.86      0.76      5400
           1       0.97      0.85      0.90      5400
           2       0.62      0.77      0.69      5400
           3       0.96      0.85      0.90      5400
           4       0.87      0.62      0.73      5400

    accuracy                           0.79     27000
   macro avg       0.82      0.79      0.80     27000
weighted avg       0.82      0.79      0.80     27000

0.790962962962963
Average Prediction Time:  0.005983996391296387
Time used:  97.76525902748108


In [22]:
from sklearn.ensemble import AdaBoostClassifier

start = time.time()
ada = AdaBoostClassifier(n_estimators=500,learning_rate=1)
ada.fit(X_train, y_train)  
y_pred = ada.predict(X_test) 

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(metrics.accuracy_score(y_test, y_pred))
get_prediction_time(ada, X_test)
print("Time used: ",time.time() - start)

[[5269    0   60    0   71]
 [   0 5136  243    0   21]
 [2038  178 3050   16  118]
 [   1   72  906 4376   45]
 [1554    0  150   72 3624]]
              precision    recall  f1-score   support

           0       0.59      0.98      0.74      5400
           1       0.95      0.95      0.95      5400
           2       0.69      0.56      0.62      5400
           3       0.98      0.81      0.89      5400
           4       0.93      0.67      0.78      5400

    accuracy                           0.79     27000
   macro avg       0.83      0.79      0.80     27000
weighted avg       0.83      0.79      0.80     27000

0.7946296296296296
Average Prediction Time:  0.0488757848739624
Time used:  793.2772843837738


In [24]:
# save the model to disk
pickle.dump(ada, open(os.path.join(model_path, "AdaBoosting_Final"), 'wb'))

### 2.4 LDA

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

start = time.time()
lda = LDA()
lda.fit(X_train, y_train)
pred=lda.predict(X_test)
print(np.unique(pred, return_counts=True))
print("Accuracy:",metrics.accuracy_score(y_test, pred))  
confmatrix = pd.DataFrame(confusion_matrix(pred, y_test, labels=[0,1]), index=[0,1])
print("\nConfusion Matrix:\n", confmatrix)
print('\n')
print(classification_report(y_test, pred, digits=3))
get_prediction_time(lda, X_test)

print("Time used: ",time.time() - start)

(array([0, 1, 2, 3, 4]), array([5564, 5541, 5325, 5470, 5100], dtype=int64))
Accuracy: 0.9548148148148148

Confusion Matrix:
       0     1
0  5295     0
1     0  5310


              precision    recall  f1-score   support

           0      0.952     0.981     0.966      5400
           1      0.958     0.983     0.971      5400
           2      0.921     0.909     0.915      5400
           3      0.969     0.981     0.975      5400
           4      0.975     0.921     0.947      5400

    accuracy                          0.955     27000
   macro avg      0.955     0.955     0.955     27000
weighted avg      0.955     0.955     0.955     27000

Average Prediction Time:  0.0
Time used:  2.549647092819214


In [23]:
# save the model to disk
pickle.dump(lda, open(os.path.join(model_path, "LDA_Final"), 'wb'))

## 3. Evaluate Model Performance

In [7]:
def get_prediction_time(model, X_test):
    start = time.time()

    for i in range(10):
        y_pred = model.predict([X_test[i]])
    end = time.time()
    print("Average Prediction Time: ",(end - start)/10)